In [2]:
import pandas as pd
import numpy as np
import catboost
print(catboost.__version__)

1.2.5


In [4]:
# grab a pre-split curated dataset that is 50% of original data
y = pd.read_csv('train50.csv', sep=',',low_memory=False)
x = y[['PCHANGE', 'GENE', 'Domain', 'ClinicalSignificance','EFFECT', 'Civic_Evidence', 'CATEGORIZATION', 'FAF', 'GNOMAD_AC', 'GNOMAD_AF', 'EXON_Rank', 'COSMIC_CNT','MVP_score', 'Civic_Drug', 'PMID_COUNT', 'KEGG']]
x = x.drop_duplicates(subset=['GENE', 'PCHANGE', 'CATEGORIZATION'], keep='first')
x.head()

,PCHANGE,GENE,Domain,ClinicalSignificance,EFFECT,Civic_Evidence,CATEGORIZATION,FAF,GNOMAD_AC,GNOMAD_AF,EXON_Rank,COSMIC_CNT,MVP_score,Civic_Drug,PMID_COUNT,KEGG
0,p.Asn1983Ser,ATM,VWFD,Benign,missense_variant,Unknown,Benign,99.73,NaN,NaN,40,NaN,NaN,1,6.0,1
1,p.Leu815Pro,ASXL1,U-box,not provided,missense_variant,Unknown,Benign,99.67,NaN,NaN,12,NaN,NaN,1,1.0,1
2,p.Phe111Leu,BCORL1,Unknown,Unknown,missense_variant,Unknown,Benign,99.77,NaN,NaN,3,NaN,NaN,1,2.0,1
3,p.Ile997Val,EP300,Unknown,Benign,missense_variant,Unknown,Benign,62.39,NaN,NaN,15,NaN,NaN,1,10.0,0
4,p.Ala2223Val,NOTCH3,Protein kinase...,Benign,missense_variant,Unknown,Benign,99.91,NaN,NaN,33,NaN,NaN,1,12.0,1


In [85]:
#here is where we will start itterating over 1 PUBLIC resource value at a time for our model
##NUMERICALS ONLY
#2 GNOMAD
#subt = x[['GENE', 'PCHANGE','CATEGORIZATION', 'GNOMAD_AF', 'GNOMAD_AC']]
#3 cosmic
#subt = x[['GENE', 'PCHANGE','CATEGORIZATION', 'COSMIC_CNT']]
#4 PMID
#subt = x[['GENE', 'PCHANGE', 'CATEGORIZATION', 'PMID_COUNT']]
#5 MVP
# reviewer requested to only use mvp on single nucleotide variants...
#l = x[x['PCHANGE'].str.contains('fs|ins|del|\\*')==False]
#subt = l[['GENE', 'PCHANGE', 'CATEGORIZATION', 'MVP_score']]
#6 
#subt = x[['GENE', 'PCHANGE','CATEGORIZATION' , 'KEGG']]
## Cats
#7 clinvar
#subt = x[['GENE', 'PCHANGE', 'CATEGORIZATION' , 'ClinicalSignificance']]
#8 
subt = x[['GENE', 'PCHANGE', 'CATEGORIZATION' , 'Civic_Evidence', 'Civic_Drug']]


In [86]:
train_df = subt.copy()

In [87]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

In [88]:
train_df.fillna(-999, inplace=True)

In [89]:
train_df.head()

,GENE,PCHANGE,CATEGORIZATION,Civic_Evidence,Civic_Drug
0,ATM,p.Asn1983Ser,Benign,Unknown,1
1,ASXL1,p.Leu815Pro,Benign,Unknown,1
2,BCORL1,p.Phe111Leu,Benign,Unknown,1
3,EP300,p.Ile997Val,Benign,Unknown,1
4,NOTCH3,p.Ala2223Val,Benign,Unknown,1


In [90]:
X = train_df.drop('CATEGORIZATION', axis=1)
y = train_df.CATEGORIZATION

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)
print('length of Xtrain and Xtest: ', len(X_train), len(X_validation))
print('length of ytrain and ytest: ', len(y_train), len(y_validation))

length of Xtrain and Xtest:  13758 3440
length of ytrain and ytest:  13758 3440


In [95]:
#categorical_names = ['GENE', 'PCHANGE'] # all numericals
#categorical_names = ['GENE', 'PCHANGE','ClinicalSignificance'] # clinvar
categorical_names = ['GENE', 'PCHANGE', 'Civic_Evidence'] # civic

cat_features = [X.columns.get_loc(i) for i in categorical_names]

In [96]:
from catboost import CatBoostClassifier
model_with_early_stop = CatBoostClassifier(
    eval_metric='AUC',
    iterations=200,
    random_seed=43,
    learning_rate=0.3,
    early_stopping_rounds=20
)
model_with_early_stop.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose=False,
    plot=False
)



In [97]:
#model_with_early_stop.save_model('gnomad_only.json', format='json')
#model_with_early_stop.save_model('cosmic_only.json', format='json')
#model_with_early_stop.save_model('pmid_only.json', format='json')
#model_with_early_stop.save_model('mvp_only.json', format='json')
#model_with_early_stop.save_model('kegg_only..json', format='json')
#model_with_early_stop.save_model('clinvar_only..json', format='json')
#model_with_early_stop.save_model('civic_only..json', format='json')